## ⚡ Zero-Shot Forecasting with EnergyTTM

This notebook performs **zero-shot energy forecasting** using a pretrained EnergyTTM model without task-specific fine-tuning.

The workflow includes:
- Loading pretrained model weights
- Preparing time-series energy data
- Running forward inference for forecasting
- Evaluating predictions using standard metrics

The objective is to assess the generalization capability of EnergyTTM on unseen buildings in a zero-shot setting.

### Getting Started

Let's import the required libraries and initializes the environment for training and evaluating TinyTimeMixer models.

It includes:
- Core Python utilities and reproducibility setup (seed control)
- PyTorch modules for model training
- HuggingFace Trainer utilities
- Time-series preprocessing and dataset loaders
- Evaluation metrics and visualization tools

The environment is configured for deterministic execution with a fixed random seed to ensure reproducible results.

In [1]:
import os
import random 
import math 
import tempfile 
import torch 
import pickle 
import logging 
import warnings
import json
import torch.nn as nn

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np 
import pandas as pd


from transformers import Trainer, TrainingArguments, set_seed, EarlyStoppingCallback
from torch.utils.data import ConcatDataset, Dataset, DataLoader


from tsfm_public.models.tinytimemixer.configuration_tinytimemixer import TinyTimeMixerConfig
from tsfm_public.models.tinytimemixer import TinyTimeMixerForPrediction

warnings.filterwarnings("ignore")
SEED = 42
set_seed(SEED)


### Evaluation Metrics

This section defines the performance metrics used to evaluate forecasting quality.

The following metrics are computed:

- **CVRMSE (Coefficient of Variation of RMSE)**  
  Measures normalized root mean squared error relative to the mean of the true values.  
  Useful for energy forecasting benchmarks where scale normalization is required.

- **MAE (Mean Absolute Error)**  
  Computes the average absolute difference between predictions and ground truth.  
  Provides an interpretable measure of average forecast deviation.

- **NRMSE (Normalized RMSE, %)**  
  Root mean squared error normalized by the mean load and expressed as a percentage.  
  Particularly suitable for daily energy series (24-hour structure assumed).

These metrics together evaluate both absolute and scale-normalized forecasting performance.


In [2]:

# metrics used for evaluation
def cal_cvrmse(pred, true, eps=1e-8):
    pred = np.array(pred)
    true = np.array(true)
    return np.power(np.square(pred - true).sum() / pred.shape[0], 0.5) / (true.sum() / pred.shape[0] + eps)

def cal_mae(pred, true):
    pred = np.array(pred)
    true = np.array(true)
    return np.mean(np.abs(pred - true))

def cal_nrmse(pred, true, eps=1e-8):
    true = np.array(true)
    pred = np.array(pred)

    M = len(true) // 24
    y_bar = np.mean(true)
    NRMSE = 100 * (1/ (y_bar+eps)) * np.sqrt((1 / (24 * M)) * np.sum((true - pred) ** 2))
    return NRMSE


## Time-Series Dataset & Scaling

This section defines preprocessing and dataset utilities for supervised forecasting.

### 🔹 Standardization
- The time series is normalized using z-score scaling:
  $
  x' = \frac{x - \mu}{\sigma}
  $
- Mean and standard deviation are stored to later **unscale predictions** back to the original energy units.

### 🔹 Sliding Window Dataset
`TimeSeriesDataset` constructs supervised training samples using a rolling window approach:

- **Backcast length** → historical input window  
- **Forecast length** → prediction horizon  
- **Stride** → step size between consecutive windows  

Each sample returns:
- `x` → past sequence (input)
- `y` → future sequence (target)

This enables efficient training for sequence-to-sequence forecasting models.


In [3]:


def standardize_series(series, eps=1e-8):
    mean = np.mean(series)
    std = np.std(series)
    standardized_series = (series - mean) / (std+eps)
    return standardized_series, mean, std

def unscale_predictions(predictions, mean, std, eps=1e-8):
    return predictions * (std+eps) + mean


class TimeSeriesDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length, stride=1):
        # Standardize the time series data
        self.data, self.mean, self.std = standardize_series(data)
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.stride = stride

    def __len__(self):
        return (len(self.data) - self.backcast_length - self.forecast_length) // self.stride + 1

    def __getitem__(self, index):
        start_index = index * self.stride
        x = self.data[start_index : start_index + self.backcast_length]
        y = self.data[start_index + self.backcast_length : start_index + self.backcast_length + self.forecast_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


### Configuring the TinyTimeMixer Model

Here, we initialize the **TinyTimeMixer forecasting model** using a structured configuration dictionary.

### 🔹 What We Do
- Create a `TinyTimeMixerConfig` from the provided `args`
- Set architectural and training parameters (context length, prediction length, layers, attention, patching, decoder options, etc.)
- Instantiate `TinyTimeMixerForPrediction` with this configuration


In [4]:
def model_config(args):

    config = TinyTimeMixerConfig(
        context_length=args["context_length"],
        patch_length=args["patch_length"],
        num_input_channels=args["num_input_channels"],
        patch_stride=args["patch_stride"],
        d_model=args["d_model"],
        num_layers=args["num_layers"],
        expansion_factor=args["expansion_factor"],
        dropout=args["dropout"],
        head_dropout=args["head_dropout"],
        mode=args["mode"][0],
        scaling=args["scaling"],
        prediction_length=args["prediction_length"],
        is_scaling=args["is_scaling"],
        gated_attn=args["gated_attn"],
        norm_mlp=args["norm_mlp"],
        self_attn=args["self_attn"],
        self_attn_heads=args["self_attn_heads"],
        use_positional_encoding=args["use_positional_encoding"],
        positional_encoding_type=args["positional_encoding_type"],
        loss=args["loss"],
        init_std=args["init_std"],
        post_init=args["post_init"],
        norm_eps=args["norm_eps"],
        adaptive_patching_levels=args["adaptive_patching_levels"],
        resolution_prefix_tuning=args["resolution_prefix_tuning"],
        frequency_token_vocab_size=args["frequency_token_vocab_size"],
        distribution_output=args["distribution_output"],
        num_parallel_samples=args["num_parallel_samples"],
        decoder_num_layers=args["decoder_num_layers"],
        decoder_d_model=args["decoder_d_model"],
        decoder_adaptive_patching_levels=args["decoder_adaptive_patching_levels"],
        decoder_raw_residual=args["decoder_raw_residual"],
        decoder_mode=args["decoder_mode"],
        use_decoder=args["use_decoder"],
        enable_forecast_channel_mixing=args["enable_forecast_channel_mixing"],
        fcm_gated_attn=args["fcm_gated_attn"],
        fcm_context_length=args["fcm_context_length"],
        fcm_use_mixer=args["fcm_use_mixer"],
        fcm_mix_layers=args["fcm_mix_layers"],
        fcm_prepend_past=args["fcm_prepend_past"], 
        init_linear=args["init_linear"],
        init_embed=args["init_embed"],

    )

    pretraining_model = TinyTimeMixerForPrediction(config)
    return pretraining_model

### 🛠️ Helper Functions for the Evaluation Pipeline

These helper functions modularize the testing workflow to improve readability, reusability, and debugging. They handle building selection, data cleaning, dataset preparation, inference, and metric computation in clearly separated steps. This structured design keeps the main evaluation loop clean and maintainable.

- **Building Selection** – Identify which building columns to evaluate (`"all"`, single, or multiple).
- **Data Cleaning** – Replace missing values using median imputation.
- **Dataset Validation** – Ensure sufficient sequence length and construct sliding-window samples.
- **Inference Execution** – Run model forward passes and collect predictions.
- **Metric Evaluation** – Unscale outputs and compute CVRMSE, NRMSE, and MAE.


In [5]:
def get_buildings_to_test(df, target_buildings):
    if target_buildings == "all":
        return list(df.columns)
    elif isinstance(target_buildings, str):
        return [target_buildings]
    elif isinstance(target_buildings, list):
        return target_buildings
    else:
        raise ValueError("target_buildings must be 'all', a string, or list")


def clean_series(energy_data):
    nan_count = np.isnan(energy_data).sum()
    if nan_count > 0:
        median_val = np.nanmedian(energy_data)
        energy_data = np.where(np.isnan(energy_data), median_val, energy_data)
        print(f"  Filled {nan_count} NaNs with median={median_val:.4f}")
    return energy_data


def create_dataset_if_valid(energy_data, args):
    min_required = args["context_length"] + args["prediction_length"]

    if len(energy_data) < min_required:
        print("   Too short, skipping...")
        return None

    dataset = TimeSeriesDataset(
        energy_data,
        args["context_length"],
        args["prediction_length"],
        args["patch_stride"]
    )

    if len(dataset) == 0:
        print("   No samples, skipping...")
        return None

    return dataset


def run_inference(model, dataset, criterion, device):
    model.eval()

    val_losses = []
    y_true_test = []
    y_pred_test = []

    for x_test, y_test in DataLoader(dataset, batch_size=1):
        x_test = x_test.unsqueeze(-1).to(device)
        y_test = y_test.to(device)

        with torch.no_grad():
            output = model(x_test)
            forecast = output.prediction_outputs.squeeze(-1)

            loss = criterion(forecast, y_test)
            if torch.isnan(loss):
                continue

            val_losses.append(loss.item())
            y_true_test.append(y_test.cpu().numpy())
            y_pred_test.append(forecast.cpu().numpy())

    if len(y_true_test) == 0:
        return None

    y_true = np.concatenate(y_true_test, axis=0)
    y_pred = np.concatenate(y_pred_test, axis=0)

    return y_true, y_pred, np.mean(val_losses)


def evaluate_predictions(y_true, y_pred, dataset):
    y_pred_unscaled = unscale_predictions(y_pred, dataset.mean, dataset.std)
    y_true_unscaled = unscale_predictions(y_true, dataset.mean, dataset.std)

    cvrmse = cal_cvrmse(y_pred_unscaled, y_true_unscaled)
    nrmse  = cal_nrmse(y_pred_unscaled, y_true_unscaled)
    mae    = cal_mae(y_pred_unscaled, y_true_unscaled)

    return cvrmse, nrmse, mae



In [6]:

def test(args, model, criterion, dataset_path, result_path, device, target_buildings="BR02"):

    os.makedirs(result_path, exist_ok=True)
    res = []

    for file_name in os.listdir(dataset_path):

        if not file_name.endswith(".parquet"):
            continue

        file_id = file_name.replace(".parquet", "")
        file_path = os.path.join(dataset_path, file_name)

        print(f"\nTesting file: {file_id}")

        df = pd.read_parquet(file_path)
        buildings_to_test = get_buildings_to_test(df, target_buildings)

        for building_col in buildings_to_test:

            if building_col not in df.columns:
                print(f" {building_col} not found, skipping...")
                continue

            print(f"   ▶ Building: {building_col}")

            energy_data = df[building_col].values.astype(np.float32)
            energy_data = clean_series(energy_data)

            dataset = create_dataset_if_valid(energy_data, args)
            if dataset is None:
                continue

            inference_output = run_inference(model, dataset, criterion, device)
            if inference_output is None:
                print("   No predictions collected, skipping...")
                continue

            y_true, y_pred, avg_loss = inference_output
            cvrmse, nrmse, mae = evaluate_predictions(y_true, y_pred, dataset)

            print(f"   CVRMSE={cvrmse:.4f}, NRMSE={nrmse:.4f}, MAE={mae:.4f}")

            res.append([file_id, building_col, cvrmse, nrmse, mae, avg_loss])

    columns = ["Dataset", "Building", "CVRMSE", "NRMSE", "MAE", "Avg_Test_Loss"]
    result_df = pd.DataFrame(res, columns=columns)

    result_csv = os.path.join(result_path, "test_results.csv")
    result_df.to_csv(result_csv, index=False)

    print("\nTesting complete!")
    print("Results saved at:", result_csv)


In [7]:
# -------------------------------------------------------
# Load Model Configuration
# -------------------------------------------------------
config_file = '../Energy-TTM/config/tinyTimeMixers.json'

# Read hyperparameters from JSON config
with open(config_file, 'r') as f:
    args = json.load(f)

# -------------------------------------------------------
# Device Setup (GPU if available, else CPU)
# -------------------------------------------------------
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# -------------------------------------------------------
# Initialize Model
# -------------------------------------------------------
# Build TinyTimeMixer model from configuration
model = model_config(args).to(device)

# Load pretrained Energy-TTM weights (zero-shot setup)
model.load_state_dict(torch.load('../Energy-TTM/Weights/energy_ttm.pth'))

# -------------------------------------------------------
# Model Statistics
# -------------------------------------------------------
# Count trainable parameters
param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Model's parameter count is:", param)

# -------------------------------------------------------
# Define Loss Function
# -------------------------------------------------------
# Mean Squared Error for forecasting evaluation
criterion = torch.nn.MSELoss()

# -------------------------------------------------------
# Run Zero-Shot Testing
# -------------------------------------------------------
# Evaluate pretrained model on forecasting dataset
test(
    args=args,
    model=model,
    criterion=criterion,
    dataset_path="../Dataset/Forecasting",
    result_path="test_results_zeroshot",
    device=device
)


Model's parameter count is: 28858

Testing file: Bareilly-1H
   ▶ Building: BR02
  Filled 9984 NaNs with median=0.1660
   CVRMSE=0.0974, NRMSE=236.3778, MAE=0.0428

Testing file: Mathura-1H
 BR02 not found, skipping...

Testing complete!
Results saved at: test_results_zeroshot/test_results.csv
